In [ ]:
import pandas as pd
import numpy as np
import time as t
import re
import itertools

from datetime import datetime as dt
from inverter_loss_calc import INV_Loss
from PF_API_Shared import get_all_plants, get_plant, get_plant_attributes, get_plant_devices, Dispatch,TimeInterval
from config import SUBSCRIPTION_KEY, CUSTOMER_ID

In [ ]:
plant = 'Cal Flats 130'
start = dt(2022,8,1)
end = dt(2022,8,2)

In [ ]:
dev_tags = [("Inverter", ["AC_POWER"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),]
#DC Current & Curtailment     
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)

inv_description = devices[devices['description'].str.contains('kW')]

inv = [x for x in inv_description["description"]]

In [ ]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

ident = [id for site,type,id,name in data.columns]

In [ ]:
ident

In [ ]:
inv

In [ ]:
data.columns = ident
list_compile = lambda a,b: a + '-' + b


for i in data.columns:
    if "INV" in i:
        mapper = (list(map(list_compile, ident,inv)))
df = data.filter(regex="INV")
df.columns = mapper
df.loc[:,"POA"] = data.filter(regex="MET").median(axis=1)

In [ ]:
inv_names = []
    
df = df[~df.index.duplicated(keep='first')]

''' Loading inv power and poa into dataframe '''

inv_power = df.filter(regex = 'INV')
all_power = df.fillna(0)

all_power.loc[:,'POA'] = df.filter(regex = 'POA').mean(axis = 1) #Manual


''' Parsing inverter metadata from the inverter name '''

inv_names = inv_power.columns.str.split('-').str[0].tolist()
n = len(inv_power.columns.str.split('-').tolist()[0])

rated_dc = []
rated_dc = [re.findall(r"[-+]?(?:\d*\.\d+|\d+)",i) for i in inv_power.columns.str.split('-').str[n-2].tolist()]
rated_dc = list(itertools.chain(*rated_dc))
rated_dc = [float(i) for i in rated_dc]

rated_ac = []
rated_ac = [re.findall(r"[-+]?(?:\d*\.\d+|\d+)",i) for i in inv_power.columns.str.split('-').str[n-1].tolist()]
rated_ac = list(itertools.chain(*rated_ac))
rated_ac = [float(i) for i in rated_ac]
# Add Inverter names not tags
''' Creating metadata table '''
metadata = pd.DataFrame({'Inv_name':inv_names,
                        'Rated_DC_KW':rated_dc,
                        'Rated_AC_KW':rated_ac})

In [ ]:
metadata